In [4]:
import pickle
import numpy as np
import re
from utils import clean

with open ("data/old/all_nba_from_oct17", 'rb') as fp:
    doc_set = pickle.load(fp)
docs = list(doc_set)

from gensim.models import Word2Vec

In [5]:
docs[0]

'The complete rosters for the 2018 NBA All-Star Game in Los Angeles were revealed Thursday after team captains LeBron James (Eastern Conference) and Stephen Curry (Western Conference) picked their squads during a conference call with league officials.\xa0The lineups and this year\'s new Jordan Brand jerseys can be viewed below:\xa0It\'s time. 🔥\n\nThe first look at the 2018 #NBAAllStar jerseys presented by @Jumpman23 https://t.co/3mRjAXGIpQTeam LeBronTeam Stephen\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0James kicked things off by making the No. 1 overall pick after he was named the All-Star Game\'s leading vote-getter. Picks subsequently alternated between the two captains for the rest of the first round until the eight remaining starters were selected.\xa0Curry\'s team then made the first pick of the second round,\xa0when all 14 reserves were eligible to come off the board.\xa0Once the draft was over, James and Curry reflected on the process and said they hope to see it televised in 2019:\xa0

In [6]:
from nltk import sent_tokenize #### rememeber this
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [7]:
def clean_sentence(sentence):
    words = [word for word in word_tokenize(sentence) if word.isalpha()]
    # convert everything to lower case
    words = [w.lower() for w in words]

    # split on caps
    words = [split_on_caps(w) for w in words]

    return " ".join(words).split()

In [10]:
# docs[0]

In [11]:
# sent_tokenize(docs[0])

['The complete rosters for the 2018 NBA All-Star Game in Los Angeles were revealed Thursday after team captains LeBron James (Eastern Conference) and Stephen Curry (Western Conference) picked their squads during a conference call with league officials.',
 "The lineups and this year's new Jordan Brand jerseys can be viewed below:\xa0It's time.",
 '🔥\n\nThe first look at the 2018 #NBAAllStar jerseys presented by @Jumpman23 https://t.co/3mRjAXGIpQTeam LeBronTeam Stephen\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0James kicked things off by making the No.',
 "1 overall pick after he was named the All-Star Game's leading vote-getter.",
 'Picks subsequently alternated between the two captains for the rest of the first round until the eight remaining starters were selected.',
 "Curry's team then made the first pick of the second round,\xa0when all 14 reserves were eligible to come off the board.",
 'Once the draft was over, James and Curry reflected on the process and said they hope to see it televised 

In [108]:
%%time
all_sentences = []
for doc in docs:
    all_sentences += [clean_sentence(sentence) for sentence in sent_tokenize(doc)]
    
len(all_sentences)

CPU times: user 22.5 s, sys: 146 ms, total: 22.6 s
Wall time: 22.8 s


In [134]:
%%time
model = Word2Vec(size=30)
model.build_vocab(all_sentences)
model.train(all_sentences, total_examples=model.corpus_count, epochs=50)

CPU times: user 2min 30s, sys: 1.03 s, total: 2min 31s
Wall time: 55.8 s


In [144]:
s = model.wv.doesnt_match('okc thunder westbrook adams love'.split())
print(s)
model.wv.most_similar('lebron')

love


[('james', 0.8014628887176514),
 ('harden', 0.7845656871795654),
 ('steph', 0.660460352897644),
 ('curry', 0.6287393569946289),
 ('captains', 0.6204112768173218),
 ('cleveland', 0.6165857315063477),
 ('pierce', 0.611995279788971),
 ('stephen', 0.6034107804298401),
 ('george', 0.5979452133178711),
 ('love', 0.596632182598114)]

In [93]:
# # persist the model to a disk
# model.save(fname)
# model = Word2Vec.load(fname)  